In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

# nn.Module로 구현하는 선형회귀

앞선 선형회귀와 다중선형회귀에서는 직접 가설과 비용함수를 코딩했지만, PyTorch에서 제공하는 함수들을 이용하여 더 쉽게 선형회귀 모델을 만들 수 있다.

예를 들어 PyTorch에는 선형 회귀 모델이 nn.Linear()라는 함수로, 또 평균 제곱오차가 nn.functional.mse_loss()라는 함수로 구현되어져 있다.

## 단순 선형회귀 구현하기

In [3]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [4]:
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)

nn.Linear(입력 차원,출력 차원) 함수는 입력 차원과 출력 차원을 입력 받는다.

nn.Linear(1,1)은 1개의 입력 값을 받아서 1개의 값을 출력한다는 뜻이다.

다중 선형 회귀 일때는 1가지가 아니고 2가지 이상의 입력 값을 받게 된다.

In [5]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


.paramters() 함수는 Model의 가중치 W와 편향 b를 출력한다.

결과 값을 보면 앞에 있는 값이 가중치의 값이고, 두번째로나오는 값이 편향의 값이다.

현재 가중치와 편향은 둘다 Random하게 설정되어 있다. 이를 학습을 해보겠다.

In [7]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train) #우리의 모델은 nn.Linear(1,1)로 정의 하였다. 이 모델에 train data를 넣어서 학습을 진행한다.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 13.103541
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


In [9]:
# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) 
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 
print(list(model.parameters()))

훈련 후 입력이 4일 때의 예측값 : tensor([[7.9989]], grad_fn=<AddmmBackward0>)
[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([0.0014], requires_grad=True)]


cost를 계산할때 F.mse_loss(prediction, y_train)를 통해 cost function인 mse를 계산하였다.

이전에는 직접 cost function의 계산을 코딩해주었지만, Pytorch에서 제공하는 함수를 이용하여 코딩하였다.

우리는 forward 연산과 backward 연산에 대해서 명확히 정의해야한다.

forward 연산은 제공한 train data를 이용하여, 우리의 예측 값을 추정하는 것이다.

따라서 코드에서 prediction = model(x_train)과 pred_y = model(new_var)이다.

반면, 학습 과정에서 Cost Function을 미분하여, 가중치와 편향을 Update해나가는 것은 backward 연산이다.

## 다중 선형 회귀 구현하기

In [12]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [13]:
# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1.
model = nn.Linear(3,1)
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.1188,  0.2937,  0.0803]], requires_grad=True), Parameter containing:
tensor([-0.0707], requires_grad=True)]


In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [15]:
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 22642.416016
Epoch  100/2000 Cost: 4.976951
Epoch  200/2000 Cost: 4.733481
Epoch  300/2000 Cost: 4.502824
Epoch  400/2000 Cost: 4.284275
Epoch  500/2000 Cost: 4.077209
Epoch  600/2000 Cost: 3.881021
Epoch  700/2000 Cost: 3.695124
Epoch  800/2000 Cost: 3.519025
Epoch  900/2000 Cost: 3.352130
Epoch 1000/2000 Cost: 3.194023
Epoch 1100/2000 Cost: 3.044178
Epoch 1200/2000 Cost: 2.902206
Epoch 1300/2000 Cost: 2.767665
Epoch 1400/2000 Cost: 2.640176
Epoch 1500/2000 Cost: 2.519374
Epoch 1600/2000 Cost: 2.404892
Epoch 1700/2000 Cost: 2.296402
Epoch 1800/2000 Cost: 2.193589
Epoch 1900/2000 Cost: 2.096157
Epoch 2000/2000 Cost: 2.003828


In [17]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 
print(list(model.parameters()))

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.9719]], grad_fn=<AddmmBackward0>)
[Parameter containing:
tensor([[0.6294, 0.6883, 0.6936]], requires_grad=True), Parameter containing:
tensor([-0.0603], requires_grad=True)]
